In [ ]:
import imdlib as imd

In [ ]:
start_yr = 2022 # give starting year from which you want to download/convert data: 1901 ownwards for rainfall, 1951 for tmax and tmin
end_yr = 2023 # give ending year upto which you want to download/convert data
variable = 'tmin' # give variable name (rain for rainfall at 0.25 deg, tmax or tmin for rainfall, min or max temperature at 1 deg resolution)
file_format = 'yearwise' # other option (None), which will assume deafult imd naming convention
file_dir = 'C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp' # this path should be same as mentioned in previous line
data = imd.open_data(variable, start_yr, end_yr,'yearwise', file_dir) # this will open the data downloaded and saved in the location mentioned in previous line

In [ ]:
if variable == 'rain':
    grid_size = 0.25 # grid spacing in deg
    y_count = 129 # no of grids in y direction
    x_count = 135 # no of grids in x direction
    x = 66.5 # starting longitude taken from control file (.ctl)
    y = 6.5 # starting latitude taken from control file (.ctl)
elif variable == 'tmax' or variable == 'tmin':
    grid_size = 1 # grid spacing in deg
    y_count = 31 # no of grids in y direction
    x_count = 31 # no of grids in x direction
    x = 67.5 # starting longitude taken from control file (.ctl)
    y = 7.5 # starting latitude taken from control file (.ctl)

#print(grid_size,x_count, y_count, x, y)
data
data.shape
np_array = data.data
#print(np_array[0,0,0])
#xr_objecct = data.get_xarray()
#type(xr_objecct)
#xr_objecct.mean('time').plot()
years_no = (end_yr - start_yr) + 1
#print(years_no)
day = 0
for yr in range(0,years_no):
    f = open("C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp"+str(start_yr+yr)+"_"+str(variable)+".csv",'w') # just change the path where you want to save csv file
    if ((start_yr+yr) % 4 == 0) and ((start_yr+yr) % 100 != 0):  # check for leap year
        days = 366
        count = yr + days
    elif ((start_yr+yr) % 4 == 0) and ((start_yr+yr) % 100 == 0) and ((start_yr+yr) % 400 == 0):
        days = 366
        count = yr + days
    else:
        days = 365
        count = yr + days

    day = day + days

    f.write("X,Y,")
    for d in range(0, days):
        f.write(str(d+1))
        f.write(",")
    f.write("\n")
    #print(np_array[364,0,0])
    for j in range(0, y_count):

        for i in range(0, x_count):

            f.write(str((i * grid_size) + x))
            f.write(",")
            f.write(str((j * grid_size) + y))
            f.write(",")
            time = 0
            for k in range(day-days, day):

                val = np_array[k,i,j]
                if val == 99.9000015258789 or val == -999:
                    f.write(str(-9999))
                    f.write(",")
                else:
                    f.write(str(val))
                    f.write(",")


            f.write("\n")
    print("File for " + str(start_yr + yr) + "_" + str(variable) + " is saved")
print("CSV conversion successful !")

Convert csv to shape file

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
def csv_to_shapefile(csv_path, shapefile_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)

# Extract the latitude and longitude columns
    geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]

    # Drop the latitude and longitude columns from the dataframe
    # df = df.drop(['X', 'Y'], axis=1)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

    # Extract date information from the column names
    for col in df.columns:
        if col.startswith('day_'):
            date_str = col.split('_')[1]
            year = date_str[:4]
            month = date_str[4:6]
            day = date_str[6:8]
            gdf[col + '_year'] = year
            gdf[col + '_month'] = month
            gdf[col + '_day'] = day

    # Save the GeoDataFrame to a shapefile
    gdf.to_file(shapefile_path, driver='ESRI Shapefile')

# Example usage
csv_path = "C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/csv2023_tmax.csv"
shapefile_path = "C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax"
csv_to_shapefile(csv_path, shapefile_path)

extract day wise with date and save in shape file

In [ ]:
import geopandas as gpd
import pandas as pd
import os
from datetime import datetime, timedelta

# Path to the input shapefile
input_shapefile = "C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax/2023_tmax.shp"

# Output directory for individual shapefiles
output_directory = "C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax/day_wise"
os.makedirs(output_directory, exist_ok=True)

# Read the shapefile
gdf = gpd.read_file(input_shapefile)

# Extract X and Y columns for lat and long
gdf['geometry'] = gpd.points_from_xy(gdf['X'], gdf['Y'])

# Replace -9999 values with NaN
gdf.replace(-9999, pd.NA, inplace=True)

# Assuming columns from the 3rd to the end are day-wise data
start_date = datetime(2023, 1, 1)

for i, col in enumerate(gdf.columns[2:], start=1):
    try:
        # Calculate the date
        current_date = start_date + timedelta(days=i-1)
        date_str = current_date.strftime("%d_%m_%Y")
        
        # Create a new GeoDataFrame with only the necessary columns
        new_gdf = gdf[['geometry', col]].copy()
        new_gdf = new_gdf.rename(columns={col: 'data'})
        
        # Remove rows with NaN values in 'data' column
        new_gdf = new_gdf.dropna(subset=['data'])
        
        # Save to a new shapefile
        output_shapefile = os.path.join(output_directory, f"{date_str}.shp")
        new_gdf.to_file(output_shapefile)

        print(f"Saved {output_shapefile}")
    except Exception as e:
        print(f"Error processing column {col}: {e}")

print("Shapefiles have been saved successfully.")

Interpolution 

In [ ]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.transform import from_origin
from scipy.interpolate import griddata

# Step 1: Read the shapefile
shapefile_path = "C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax/day_wise/01_01_2023.shp"
gdf = gpd.read_file(shapefile_path)

# Extract x, y coordinates and the value to interpolate
x = gdf.geometry.x
y = gdf.geometry.y
z = gdf['data']  # Replace with the name of the column to interpolate

# Define the pixel size
pixel_size = 0.001000  # Replace with your desired pixel size

# Step 2: Define the grid for interpolation with specified pixel size
x_min, x_max = min(x), max(x)
y_min, y_max = min(y), max(y)

# Calculate number of pixels along X and Y dimensions
num_pixels_x = int((x_max - x_min) / pixel_size) + 1
num_pixels_y = int((y_max - y_min) / pixel_size) + 1

# Create the grid with the specified number of pixels
grid_x, grid_y = np.meshgrid(np.linspace(x_min, x_max, num_pixels_x),
                             np.linspace(y_min, y_max, num_pixels_y))

# Perform cubic interpolation
grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

# Reverse the y-axis to correct orientation
grid_z = np.flipud(grid_z)

# Step 3: Save the interpolated surface as a TIFF file with compression and same extent
transform = from_origin(x_min, y_max, pixel_size, pixel_size)

output_tiff_path = 'C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax/inter_data/01_01_2023_1.tif'

with rasterio.open(
    output_tiff_path,
    'w',
    driver='GTiff',
    height=grid_z.shape[0],
    width=grid_z.shape[1],
    count=1,
    dtype=grid_z.dtype,
    crs=gdf.crs,  # Use the CRS from the shapefile
    transform=transform,
    compress='DEFLATE',  # Apply DEFLATE compression
    zlevel=9  # Set high compression level
) as new_dataset:
    new_dataset.write(grid_z, 1)

yearly data download

In [ ]:
import imdlib as imd
start_yr = 2022
end_yr = 2024
# other options are ('tmin'/ 'tmax')
variable = 'tmax' 
file_dir = (r'C:/Users/Bandesh/Satsure/others/Yes_tech/Barghar_dist/imd_temp/new_test/2023_tmax/') #Path to save the files
imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=file_dir)

The Daily Real Time Data available,

rain = Daily Rainfall data
raingpm = Daily Merged Satellite Guage Rainfall (GPM) data
tmax = Daily Maximum Temperature data
tmin = Daily Minimum Temperature Data
tmaxone = Daily Maximum Temperature data (1.0 degree)
tminone = Daily Minimum Temperature Data (1.0 degree)
Installation

In [ ]:
from imddaily import imddaily
data = imddaily.get_data('tmin', '2023-03-01', '2023-12-01', 'C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp/2023_tmin/')
data.to_geotiff('C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp/2023_tmin/')

In [ ]:
from imddaily import imddaily
data = imddaily.get_data('rain', '2023-03-01', '2023-12-01', 'C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp/2023_tmin/')
data.to_geotiff('C:/Users/Bandesh/Satsure/others/Yes_tech/imd_temp/2023_tmin/')